In [3]:
import os

In [4]:
%pwd

'c:\\projects\\end-to-end-mlprojects\\wine_quality_end_to_end_prediction\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\projects\\end-to-end-mlprojects\\wine_quality_end_to_end_prediction'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [8]:
from wine_quality_end_to_end_Project.constants import *
from wine_quality_end_to_end_Project.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:

    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
    ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from wine_quality_end_to_end_Project import logger
from wine_quality_end_to_end_Project.utils.common import get_size
from wine_quality_end_to_end_Project.entity.config_entity import DataIngestionConfig
from pathlib import Path

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config



    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exxists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extract the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-27 11:08:04,238: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-27 11:08:04,240: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-27 11:08:04,240: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-27 11:08:04,240: INFO: common: created directory at: artifacts]
[2025-01-27 11:08:04,254: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-27 11:08:04,254: INFO: 119303750: File already exxists of size: ~ 857 KB]


BadZipFile: File is not a zip file